In [7]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import base64 as b64
from html_to_markdown import convert_to_markdown
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [8]:
headers = {
  'accept': 'application/json, text/javascript, */*; q=0.01',
#   'accept-encoding': 'gzip, deflate, br, zstd',
  'cookie': os.getenv('COOKIE'),
  'x-requested-with': 'XMLHttpRequest'
}

### getting all post urls between jan 1 - april 14

In [9]:
post_urls = []

for i in range(15):
    url = f"https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34.json?page={i}"
    r = requests.get(url, headers=headers)

    posts = r.json()['topic_list']['topics']

    for post in posts:
        if datetime(2025, 1, 1) <= datetime.strptime(post['created_at'], "%Y-%m-%dT%H:%M:%S.%fZ") <= datetime(2025, 4, 14):
            post_urls.append("https://discourse.onlinedegree.iitm.ac.in/t/"+str(post["id"])+"/posts.json")

In [10]:
len(post_urls) 

117

### define a function to let an VLM describe images

In [11]:
def get_image_description(img_url):
    r = requests.post("http://localhost:11434/api/generate", json={
        "model": "minicpm-v:8b-2.6-q8_0",
        "prompt": "Respond with a concise caption describing image",
        "stream": False,
        "images" : [b64.b64encode(requests.get(img_url).content).decode('utf-8')]
    })
    
    if r.status_code != 200:
        print("failed to get image description for " + img_url)        
        print("error: " + r.text)

    return r.json()['response']

### retrieving the post contents and saving them as markdown  

In [13]:
for url in post_urls:
    replies = []
    
    post = requests.get(url, headers=headers).json()

    for reply in post['post_stream']['posts']:
        soup = BeautifulSoup("@" + reply["display_username"] + " said: " + reply["cooked"] , "html.parser")

        # add image description later
        # for img in soup.find_all("img"):
        #         url = img.attrs['src']
        #         description = get_image_description(url)
        #         img.replace_with(f"![{description}]({url})") 
                
        replies.append(str(soup))

    # Include slug in filename for easy URL reconstruction
    topic_id = post["id"]
    topic_slug = post["post_stream"]["posts"][0]["topic_slug"]
    with open(f"discourse_content/{topic_id}-{topic_slug}.md", "w+") as file:
        file.write(convert_to_markdown("\n".join(replies)))